In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-20 13:46:37.261839: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 13:46:37.469544: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
reload(config)
reload(processed)
tokenized_data,loaded_data,raw_data = processed.tokenize_and_load_datasets()


Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

Map:   0%|          | 0/69615 [00:00<?, ? examples/s]

Map:   0%|          | 0/7735 [00:00<?, ? examples/s]

In [3]:
dataset= raw_data['source']
dataset

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 8335
    })
})

In [4]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
init(model)

In [6]:
from transformers import AdamW
from adapters import AdapterConfig,LoRAConfig
from adapters.composition import Stack

adapter_name= "lora-source-mlm"
adapter_config = LoRAConfig(r=8, alpha=16)
model.add_adapter(adapter_name, config=adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter(adapter_name)
model.active_adapters = adapter_name

fn.print_trainable_parameters(model)

trainable params: 739584 || all params: 67132986 || trainable%: 1.1016700493554688


In [7]:
model.adapter_summary(as_dict=True)

[{'name': 'lora-source-mlm',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': True,
  '%param': 0.22219650503413957},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [8]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(dataset,tokenizer)
tokenized_dataset

Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 8335
    })
})

In [9]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset['train'][444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [10]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [11]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 24810
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2741
    })
})

In [12]:
print(tokenizer.decode(lm_datasets['test'][1]["input_ids"]))
print(tokenizer.decode(lm_datasets['test'][1]['labels']))
print(len(lm_datasets['test'][-1]['labels']))


things for um helping people uh survive a divorce uh the road less traveled was probably one of my favorites have you read that [SEP] the road less traveled is a book that can help people survive a divorce. [SEP] [CLS] it's not a violent overthrow of the people in washington [SEP] washington is impenetrable. [SEP] [CLS] buy a new spark plug or something along those lines but fortunately it rains and you uh do not have to go out and buy the spark plug no but we've had an unusually uh uh warm [SEP] you need new spark plugs, because your engine has been running poorly. [SEP] [CLS] hard for me because you know it
things for um helping people uh survive a divorce uh the road less traveled was probably one of my favorites have you read that [SEP] the road less traveled is a book that can help people survive a divorce. [SEP] [CLS] it's not a violent overthrow of the people in washington [SEP] washington is impenetrable. [SEP] [CLS] buy a new spark plug or something along those lines but fortu

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets['train'][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] and i mean majority i'd say at least [MASK] usually [MASK] [MASK] a week he [MASK] his lunch [SEP] since he eats lunch outside of work, he never takes a lunch. [SEP] [CLS] because it's yeah and it's so it's [MASK] to pull itself along and my husband [MASK] doesn't bother [MASK] and he i'[MASK] got out and mowed lightning him one day just to be nice so he wouldn [MASK] t have to do it when he got home and i said what is with this mower i can't [MASK] push it around the yard [SEP] i wanted to mow the lawn for my husband. [SEP]'

'>>> [CLS] if [MASK] you know [MASK] it was like my my grandmother had thirteen kids so they all you [MASK] paid their month which [MASK] better but see my husband's [MASK] only [MASK] [SEP] my grandma only had one kid [MASK] [SEP] [CLS] they don't either [MASK] well the first one i joined they [MASK] [MASK] [MASK] business [MASK]hong transferred it to another [MASK] that was really far away [SEP] [MASK] place never went out of business, they're [MASK

In [14]:
reload(fn)
samples = [lm_datasets['train'][i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [MASK] i mean so i'd say [MASK] least two usually three times a week he [MASK] his lunch [SEP] since he eats lunch outside [MASK] work, he never takes a lunch. [SEP] [CLS] [MASK] it [MASK] s [MASK] and it's so it's supposed to pull itself along and my husband it doesn't bother him and he i'm got [MASK] and mowed for him one day just to be nice so [MASK] wouldn't have to do it when he got home and i said what is with this mower i [MASK] [MASK] t even push it around the yard [SEP] i wanted to mow the [MASK] for my husband. [SEP]'

'>>> [CLS] if she [MASK] know since [MASK] was like my my grandmother had thirteen [MASK] [MASK] they all you know paid [MASK] month which was better but [MASK] my husband's a only child [SEP] my grandma only had one kid. [SEP] [CLS] they don't either well well the first one i joined [MASK] [MASK] out of business and they transferred it to another [MASK] that was really far away [SEP] the place never went out of business [MASK] they [MASK] [MASK] [M

In [15]:

reload(fn)
trainer = fn.train_mlm_model(model,"lora-source-mlm",data_collator,tokenizer, lm_datasets['train'],lm_datasets['test'])

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [16]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 31.60


In [17]:
trainer.train()

  0%|          | 0/7760 [00:00<?, ?it/s]

{'loss': 2.7041, 'learning_rate': 9.621212121212123e-05, 'epoch': 1.0}


  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 2.3539438247680664, 'eval_runtime': 2.8869, 'eval_samples_per_second': 949.457, 'eval_steps_per_second': 29.79, 'epoch': 1.0}
{'loss': 2.3887, 'learning_rate': 8.553719008264463e-05, 'epoch': 2.0}


  0%|          | 0/86 [00:00<?, ?it/s]

...

{'eval_loss': 2.130790948867798, 'eval_runtime': 2.8685, 'eval_samples_per_second': 955.553, 'eval_steps_per_second': 29.981, 'epoch': 9.0}
{'loss': 2.2018, 'learning_rate': 1.5151515151515152e-07, 'epoch': 9.99}


  0%|          | 0/86 [00:00<?, ?it/s]

Overwriting existing adapter 'lora-source-mlm'.


{'eval_loss': 2.1180198192596436, 'eval_runtime': 2.8539, 'eval_samples_per_second': 960.448, 'eval_steps_per_second': 30.134, 'epoch': 10.0}
{'train_runtime': 526.9221, 'train_samples_per_second': 470.848, 'train_steps_per_second': 14.727, 'train_loss': 2.300284755844431, 'epoch': 10.0}


TrainOutput(global_step=7760, training_loss=2.300284755844431, metrics={'train_runtime': 526.9221, 'train_samples_per_second': 470.848, 'train_steps_per_second': 14.727, 'train_loss': 2.300284755844431, 'epoch': 10.0})

In [18]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 8.39


In [19]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [20]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

In [21]:
masked_sentence = "I cancelled my [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my flight.
>>> i cancelled my appointment.
>>> i cancelled my plans.
>>> i cancelled my meeting.
>>> i cancelled my vacation.
